## 2. Multivariate Regression

In [1]:
from pylab import *
%matplotlib inline
from sklearn.cross_validation import KFold
from sklearn.preprocessing import *
from sklearn.linear_model import LinearRegression
from datetime import datetime

###Function to load the data from data file

In [2]:
def getData(fileName):
    strData=[]
    with open(fileName,'r') as f:
        for line in f:
            if line.strip().find('#'):
                strData.append(line.strip().split(" "))
    data = np.array(strData, dtype='|S25').astype(np.float64)
    return data

###Function to calculate Mean Square Error (MSE)

In [41]:
def calcErr(Y_est, Y):
    m = len(Y)
    err=np.zeros(m)
    Yavg = np.mean(Y)
    for i in range(m):
        err[i] = ((Y_est[i] - Y[i])**2)/m 
    return np.sum(err)/m 

###Function to map the dataset to higher order polynomial and fit a regression model

In [64]:
def poly_fit(training_data, test_data, degree):   
    m_train = len(training_data)
    X_train = training_data[:,:2].reshape((m_train,2))
    Y_train = training_data[:,2].reshape((m_train,1))
        
    m_test = len(test_data)
    X_test = test_data[:,:2].reshape((m_test,2))
    Y_test = test_data[:,2].reshape((m_test,1))
    
    poly = PolynomialFeatures(degree)
    Z_train = poly.fit_transform(X_train)
    
    theta = np.dot(np.linalg.pinv(Z_train),Y_train)
    Y_train_est = np.dot(Z_train, theta)
    
    Z_test = poly.fit_transform(X_test)
    Y_test_est = np.dot(Z_test, theta)
    
    err_train = calcErr(Y_train_est,Y_train)
    err_test = calcErr(Y_test_est,Y_test)
    
    err = [err_train, err_test]
    
    return err

###Function to fit a polynomial model using Python library functions

In [40]:
def poly_lm_fit(training_data, test_data, degree):     
    m_train = len(training_data)
    X_train = training_data[:,:2].reshape((m_train,2))
    Y_train = training_data[:,2].reshape((m_train,1))
        
    m_test = len(test_data)
    X_test = test_data[:,:2].reshape((m_test,2))
    Y_test = test_data[:,2].reshape((m_test,1))
    
    poly = PolynomialFeatures(degree)
    Z_train = poly.fit_transform(X_train)
    
    lm = LinearRegression()
    lm.fit(Z_train,Y_train)    
    Y_train_est = lm.predict(Z_train)
    
    Z_test = poly.fit_transform(X_test)
    Y_test_est = lm.predict(Z_test)
    
    err_train = calcErr(Y_train_est,Y_train)
    err_test = calcErr(Y_test_est,Y_test)
    
    err = [err_train, err_test]
    
    return err

###Function to normalize the dataset for Iterative Method

In [6]:
def normaliza(data):
    
    [nrows, ncols] = data.shape
    
    col_mean = np.zeros(ncols).reshape(ncols, 1)
    col_sd = np.zeros(ncols).reshape(ncols, 1)
    normalized_data = np.zeros(data.shape)
    
    for i in range(ncols):
        col_mean[i] = np.mean(data[:,i])
        col_sd = np.std(data[:,i])
        nomarlized_data[:,i] = [(data[:,i][j] - col_mean[i])/col_sd for j in range(nrows)]
    
    return normalized_data

###Function to perform Stochastic Gradient Descent Method

In [38]:
def iterative_method(training_data, test_data, degree, theta_0, learning_rate):
    
    import random
    
    m_train = len(training_data)
    X_train = training_data[:,:2].reshape((m_train,2))
    Y_train = training_data[:,2].reshape((m_train,1))
    
    m_test = len(test_data)
    X_test = test_data[:,:2].reshape((m_test,2))
    Y_test = test_data[:,2].reshape((m_test,1))
    
    poly = PolynomialFeatures(degree)
    Z_train = poly.fit_transform(X_train)
    
    n = Z_train.shape[1]
    theta_old = np.zeros(n).reshape(n,1)
    theta_old.fill(theta_0)
    theta = np.zeros(n)
    
    randomIdx = np.arange(m_train)
    random.shuffle(randomIdx)
    X_train = X_train[randomIdx]
    Y_train= Y_train[randomIdx]
    
    for i in range(m_train):
        for j in range(m_train):
            step = learning_rate * (np.dot(Z_train[j], theta_old) - Y_train[j]) * Z_train[j].T
            theta = theta_old - step.reshape(n,1)
            eps = max(theta_old-theta)
            if eps <= 0.0000000001:
                break
            theta_old = theta
        
    Y_train_est = np.dot(Z_train, theta)
    
    Z_test = poly.fit_transform(X_test)
    Y_test_est = np.dot(Z_test, theta)
    
    err_train = calcErr(Y_train_est,Y_train)
    err_test = calcErr(Y_test_est,Y_test)
    
    err = [err_train, err_test]
    
    return err

###Function to implement Guassian Kernel function

In [39]:
def gaussian_kernel(X, Y, sigma):
    m = len(X)
    G = np.zeros((m,m))
    distance = np.zeros((m,m))
    
    for i in range(len(X)):
        for j in range(len(Y[0])):
            for k in range(len(Y)):
                distance[i,j] += (X[i,k] - Y[k,j])**2
            G[i,j] = np.exp((-1/2)*(distance[i,j]/sigma**2))
    #print G.shape
    return G

###Function to solve the Dual Problem

In [37]:
def dual_regression(training_data, test_data,degree):
    m_train = len(training_data)
    X_train = training_data[:,:2].reshape((m_train,2))
    Y_train = training_data[:,2].reshape((m_train,1))
        
    m_test = len(test_data)
    X_test = test_data[:,:2].reshape((m_test,2))
    Y_test = test_data[:,2].reshape((m_test,1))
    
    poly = PolynomialFeatures(degree)
    
    Z_train = poly.fit_transform(X_train)    
    G_train = gaussian_kernel(Z_train, Z_train.T, 0.1)
    alpha = solve(G_train, Y_train)
    Y_train_est = np.dot(alpha.T, G_train)
    Y_train_est = Y_train_est.T
    
    Z_test = poly.fit_transform(X_test)
    G_test = gaussian_kernel(Z_train, Z_test.T, 0.01)    
    Y_test_est = np.dot(alpha.T, G_test)
    Y_test_est = Y_test_est.T
    
    err_train = calcErr(Y_train_est,Y_train)
    err_test = calcErr(Y_test_est,Y_test)
    
    err = [err_train, err_test]
    
    return err

###Function to load real data

In [73]:
def getRealData():
    from sklearn import datasets
    data = datasets.load_boston() 
    #data = datasets.load_diabetes()
    X = data['data']
    Y = data['target']
    Y = Y.reshape(len(data['target']), 1)
    dataset = np.hstack((X, Y))
    return dataset

###Master function that invokes all the functions

In [49]:
def run(fileName, K, degree, theta, alpha):
    
    if fileName=='real':
        dataset = getRealData()
    else:
        dataset = getData(fileName)
    
    
    #Slicing to use reduced data
    dataset = dataset[0:500]
    
    #K = 2
    #degree = 6
    #theta_0 = 0.3
    #learning_rate = 0.1
    
    CV_idx = KFold(len(dataset), n_folds=K)
    err_poly_train = np.zeros(K)
    err_poly_test = np.zeros(K)
    err_poly_lm_train = np.zeros(K)
    err_poly_lm_test = np.zeros(K)
    err_dual_train = np.zeros(K)
    err_dual_test = np.zeros(K)
    err_iter_train = np.zeros(K)
    err_iter_test = np.zeros(K)
    time_taken_primal = []
    time_taken_dual = []
    
    normalized_dataset = normalize(dataset.reshape((len(dataset),len(dataset[0]))))
    
    i = 0
    for train_idx, test_idx in CV_idx:
        start_primal = datetime.now()
        [err_poly_train[i], err_poly_test[i]] = poly_fit(dataset[train_idx], dataset[test_idx], degree)
        stop_primal = datetime.now()
        
        [err_poly_lm_train[i], err_poly_lm_test[i]] = poly_lm_fit(dataset[train_idx], dataset[test_idx], degree)
        
        start_dual = datetime.now()
        [err_dual_train[i], err_dual_test[i]] = dual_regression(dataset[train_idx], dataset[test_idx],degree)
        stop_dual = datetime.now()
        time_taken_primal.append(stop_primal - start_primal)
        time_taken_dual.append(stop_dual - start_dual)
        
        [err_iter_train[i], err_iter_test[i]] = iterative_method(
            normalized_dataset[train_idx], normalized_dataset[test_idx],degree, theta, alpha)
        
        i+=1 
        
    t_primal = 0
    t_dual = 0
    
    for i in range(K):
        t_primal += time_taken_primal[i].total_seconds()
        t_dual += time_taken_dual[i].total_seconds()
            
    print "\t\t\t\t\t <Training Error> <Test Error>"
    print "Polynomial Model Errors: \t\t", np.mean(err_poly_train), np.mean(err_poly_test)
    print "Python Polynomial Model Errors: \t", np.mean(err_poly_lm_train), np.mean(err_poly_lm_test)
    
    print "\nExplicit Model Errors: \t\t\t", np.mean(err_poly_train), np.mean(err_poly_test)
    print "Iterative Method Errors: \t\t", np.mean(err_iter_train), np.mean(err_iter_test)
    
    print "\nPrimal Model Errors: \t\t\t", np.mean(err_poly_train), np.mean(err_poly_test), "<Time Elapsed: ", t_primal/K, "s >"
    print "Dual Model Errors: \t\t\t", np.mean(err_dual_train), np.mean(err_dual_test), "<Time Elapsed: ", t_dual/K, "s >"

###Main function that performs Multivariate Regression

In [75]:
def main():
    realData = 'real'
    #fileName = realData
    fileName = 'mvar-set4.dat'
    
    run(fileName, K = 10, degree = 5, theta = 0.3, alpha = 0.01) 

    
if __name__ == '__main__':
    main()

					 <Training Error> <Test Error>
Polynomial Model Errors: 		0.00342084100209 0.0342322596925
Python Polynomial Model Errors: 	0.00342084100209 0.0342322596925

Explicit Model Errors: 			0.00342084100209 0.0342322596925
Iterative Method Errors: 		0.000696065985947 0.00633241469444

Primal Model Errors: 			0.00342084100209 0.0342322596925 <Time Elapsed:  0.0161 s >
Dual Model Errors: 			1.57225102888e-34 0.032718579732 <Time Elapsed:  15.675 s >
